In [1]:
import warnings

warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
import seaborn as sns
import scanpy as sc
import pandas as pd
import numpy as np
import random
import sc_toolbox

import rpy2.rinterface_lib.callbacks
import anndata2ri
import logging

from rpy2.robjects import pandas2ri
from rpy2.robjects import r

sc.settings.verbosity = 0
rpy2.rinterface_lib.callbacks.logger.setLevel(logging.ERROR)

pandas2ri.activate()
anndata2ri.activate()

%load_ext rpy2.ipython  

In [2]:
%%R
library(edgeR)
library(MAST)

Loading required package: limma
Loading required package: SingleCellExperiment
Loading required package: SummarizedExperiment
Loading required package: MatrixGenerics
Loading required package: matrixStats

Attaching package: 'MatrixGenerics'

The following objects are masked from 'package:matrixStats':

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2

In [3]:
adata_obs_meta = pd.read_csv('./adata_obs_meta.csv').set_index('obs_id')
adata_train = pd.read_parquet('./adata_train.parquet')

# parquet to matrix
adata_xdf = adata_train.pivot_table(index="obs_id", columns="gene", values="count").fillna(0)

# matrix to adata
adata = sc.AnnData(
    X=adata_xdf,
    obs=adata_obs_meta.loc[adata_xdf.index],
    var=pd.DataFrame(adata_xdf.columns, index=adata_xdf.columns)
)

# Save h5ad (~40G)
adata.write_h5ad('adata.h5ad')


MemoryError: Unable to allocate 3.10 GiB for an array with shape (416442312,) and data type int64

In [ ]:
adata

cell_type             sm_name sm_lincs_id  \
0              NK cells        Clotrimazole    LSM-5341   
1          T cells CD4+        Clotrimazole    LSM-5341   
2          T cells CD8+        Clotrimazole    LSM-5341   
3    T regulatory cells        Clotrimazole    LSM-5341   
4              NK cells  Mometasone Furoate    LSM-3349   
..                  ...                 ...         ...   
609  T regulatory cells        Atorvastatin    LSM-5771   
610            NK cells           Riociguat   LSM-45758   
611        T cells CD4+           Riociguat   LSM-45758   
612        T cells CD8+           Riociguat   LSM-45758   
613  T regulatory cells           Riociguat   LSM-45758   

                                                SMILES  control      A1BG  \
0               Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.104720   
1               Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.915953   
2               Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False -0.387721   
3               Clc1ccccc1C(c1ccccc1)(c1ccccc1)n1ccnc1    False  0.232893   
4    C[C@@H]1C[C@H]2[C@@H]3CCC4=CC(=O)C=C[C@]4(C)[C...    False  4.290652   
..                                                 ...      ...       ...   
609  CC(C)c1c(C(=O)Nc2ccccc2)c(-c2ccccc2)c(-c2ccc(F...    False -0.014372   
610  COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...    False -0.455549   
611  COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...    False  0.338168   
612  COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...    False  0.101138   
613  COC(=O)N(C)c1c(N)nc(-c2nn(Cc3ccccc3F)c3ncccc23...    False -0.757116   

     A1BG-AS1       A2M   A2M-AS1     A2MP1  ...      ZUP1      ZW10  \
0   -0.077524 -1.625596 -0.144545  0.143555  ... -0.227781 -0.010752   
1   -0.884380  0.371834 -0.081677 -0.498266  ... -0.494985 -0.303419   
2   -0.305378  0.567777  0.303895 -0.022653  ... -0.119422 -0.033608   
3    0.129029  0.336897  0.486946  0.767661  ...  0.451679  0.704643   
4   -0.063864 -0.017443 -0.541154  0.570982  ...  0.758474  0.510762   
..        ...       ...       ...       ...  ...       ...       ...   
609 -0.122464 -0.456366 -0.147894 -0.545382  ... -0.549987 -2.200925   
610  0.188181  0.595734 -0.100299  0.786192  ... -1.236905  0.003854   
611 -0.109079  0.270182 -0.436586 -0.069476  ...  0.077579 -1.101637   
612 -0.409724 -0.606292 -0.071300 -0.001789  ...  0.005951 -0.893093   
613  0.085910 -0.730025 -1.367801 -0.695944  ...  0.232343 -2.247816   

       ZWILCH     ZWINT      ZXDA      ZXDB      ZXDC    ZYG11B       ZYX  \
0   -0.023881  0.674536 -0.453068  0.005164 -0.094959  0.034127  0.221377   
1    0.304955 -0.333905 -0.315516 -0.369626 -0.095079  0.704780  1.096702   
2   -0.153123  0.183597 -0.555678 -1.494789 -0.213550  0.415768  0.078439   
3    0.015468 -0.103868  0.865027  0.189114  0.224700 -0.048233  0.216139   
4    0.607401 -0.123059  0.214366  0.487838 -0.819775  0.112365 -0.122193   
..        ...       ...       ...       ...       ...       ...       ...   
609  0.359806  1.073983  0.356939 -0.029603 -0.528817  0.105138  0.491015   
610 -0.197569 -0.175307  0.101391  1.028394  0.034144 -0.231642  1.023994   
611  0.457201  0.535184 -0.198404 -0.005004  0.552810 -0.209077  0.389751   
612 -1.003029 -0.080367 -0.076604  0.024849  0.012862 -0.029684  0.005506   
613 -0.346036 -0.919567 -1.131372 -0.120252 -0.064537 -0.603280 -0.098041   

        ZZEF1  
0    0.368755  
1   -0.869887  
2   -0.259365  
3   -0.085024  
4    0.676629  
..        ...  
609 -0.979951  
610 -0.064760  
611 -0.337082  
612 -1.733112  
613 -0.750681  

[614 rows x 18216 columns]